The data is related with direct marketing campaigns of a Portuguese banking institution. 
   The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, 
   in order to access if the product (bank term deposit) would be (or not) subscribed. 

> Number of Instances: 41188

> Number of Attributes: 20 + output attribute.


The classification goal is to predict if the client will subscribe a term deposit (variable y).


# Attribute information:
   Input variables:
  
   ## Bank client data:
   
   1 - age (numeric)
  
   2 - Job : type of job (categorical: 'admin.', 'blue-collar', 'entrepreneur', 'housemaid', 'management', 'retired', 'self-employed', 'services', 'student', 'technician', 'unemployed', 'unknown')
   
   3 - marital : marital status (categorical: 'divorced', 'married', 'single', 'unknown' ; note: 'divorced' means divorced or widowed)
  
   4 - Education (categorical: 'basic.4y', 'basic.6y', 'basic.9y', 'high.school', 'illiterate', 'professional.course', 'university.degree', 'unknown')

   
   5 - Default: has credit in default? (categorical: 'no', 'yes', 'unknown')
  
   6 - Housing: has housing loan? (categorical: 'no', 'yes', 'unknown')
  
   7 - Loan: has personal loan? (categorical: 'no', 'yes', 'unknown')
  
   ## Related with the last contact of the current campaign:
   
   8 - contact: contact communication type (categorical: "unknown","telephone","cellular") 
  
   9 - Dayofweek: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
  
   10 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")
  
   11 - Duration: last contact duration, in seconds (numeric). 
   
Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

  
   ## other attributes:
  
  12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
  
  13 - Pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
  
  14 - previous: number of contacts performed before this campaign and for this client (numeric)

  15 - poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")
  
  ## Social and economic context attributes:
  
  16 - Emp.var.rate: employment variation rate - quarterly indicator (numeric)
  
  17 - Cons.price.idx: consumer price index - monthly indicator (numeric)
  
  18 - Cons.conf.idx: consumer confidence index - monthly indicator (numeric)
  
  19 - Euribor3m: euribor 3 month rate - daily indicator (numeric)
  
  20 - Nr.employed: number of employees - quarterly indicator (numeric)

# Output variable (desired target):

  21 - y - has the client subscribed a term deposit? (binary: "yes","no")

In [ ]:
#importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
path = '../input/bank-marketing/bank-additional-full.csv'
df  = pd.read_csv(path, sep=';')
df.head()

In [ ]:
df.info()

In [ ]:
print(df.y.value_counts())

dfgrouped = df.groupby('y')

In [ ]:
for type,data in dfgrouped:
  display(type)
  display(data)

In [ ]:
def feature_perc(feature, groupby= 'yes'):

  count = dfgrouped.get_group(groupby)[feature].value_counts()
  total_count = df[feature].value_counts()[count.index]

  perc = (count/total_count)*100
  return perc 


def plot_barh(array,incrementer, bias,ax= None, text_color ='blue', palette_style = 'darkgrid',palette_color = 'RdBu'):

  sns.set_style(palette_style)
  sns.set_palette(palette_color)
    
  sns.barplot(x= array, y= array.index, ax=ax)
  #plt.barh(array.index, width = array.values, height = .5)
  plt.yticks(np.arange(len(array)))
  plt.xticks( range(0, round(max(array)) +bias, incrementer ))

  for index, value in enumerate(array.values):
    plt.text(value +.5, index, s= '{:.1f}%'.format(value), color = text_color)

  #plt.show()
  return plt

In [ ]:
object_feature_list = list(df.dtypes[df.dtypes == 'object'].index)

for feature in object_feature_list[:-1]:
    
    feature_perct =  feature_perc(feature)
    plt.title('Success rate by {}'.format(feature))
    plot_barh(feature_perct.sort_values(ascending= False),5,10, text_color = 'blue')
    plt.show()

In [ ]:
int_feature_list = list(df.dtypes[df.dtypes != 'object'].index)

for feature in int_feature_list:
    
    fig, ax = plt.subplots(1,2, figsize= (12,6))
    sns.boxplot(x='y', y=feature, data =df, showmeans=True, ax = ax[0])
    sns.distplot(df[feature], ax = ax[1], kde= False)
    ax[0].set_title('{} variation'.format(feature))
    plt.show()


In [ ]:
#df.drop(df[df['duration']>4000].index,  inplace =True)

In [ ]:
#df.drop( df[df.previous>100].index, inplace=True)

In [ ]:
df['y'] = df.y.apply(lambda x:0 if x=='no' else 1)
display(df.head())

In [ ]:
df2 = pd.get_dummies(df)
display(df2.head())

In [ ]:
display(df2.corr()['y'].sort_values(ascending= False))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df2.drop('y',axis=1),
                                                    df2['y'],
                                                    test_size=.25, random_state = 42,
                                                    stratify= df2['y'])

In [ ]:
stsc = StandardScaler()
s_X_train = stsc.fit_transform(X_train)
s_X_test = stsc.transform(X_test)

In [ ]:
model = LogisticRegression()
#model = SVC()
#model = RandomForestClassifier()
model.fit(s_X_train,y_train)

In [ ]:
model.score(s_X_train,y_train)

In [ ]:
model.score(s_X_test,y_test)

In [ ]:
confusion_matrix(y_train, model.predict(s_X_train))

In [ ]:
print(classification_report(y_train, model.predict(s_X_train)))

In [ ]:
confusion_matrix(y_test, model.predict(s_X_test))

In [ ]:
print(classification_report(y_test, model.predict(s_X_test)))

## ML Model Building

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline

In [ ]:
X_train, X_test, y_train, y_test  = train_test_split(
                                    df2.drop('y', axis=1), df2['y'], test_size=.25, random_state = 0, stratify= df2['y'])

In [ ]:
#model = LogisticRegression()
#model = DecisionTreeClassifier()
model = RandomForestClassifier()
#model = SVC

In [ ]:
steps = [('scaler', StandardScaler()),
         ('model', model)]
pipeline = Pipeline(steps)
pipeline.fit(X_train,y_train)

In [ ]:
y_pred = pipeline.predict(X_test)
print(y_pred)

In [ ]:
print('confusion matrix for training data:')
print(confusion_matrix(y_train, pipeline.predict(X_train)))
print('==============================')
print('confusion matrix for testing data:')
print(confusion_matrix(y_test, pipeline.predict(X_test)))

In [ ]:
print('classification report for training data:')
print(classification_report(y_train, pipeline.predict(X_train)))
print('==============================')
print('classification report for testing data:')
print(classification_report(y_test, pipeline.predict(X_test)))

In [ ]:
accuracy_score(y_test, pipeline.predict(X_test))

## Still not completed.